In [ ]:
import cv2
import glob
import numpy as np


import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from importlib import reload
from IPython.display import display
from matplotlib.pyplot import figure

In [ ]:
def showImage(img,imfg = True):
    fig,ax = plt.subplots(1,figsize=(25,25))
    ax.imshow(img,cmap = 'gray')
    return ax

In [ ]:
img = cv2.cvtColor(cv2.imread('../data/TRACKB_training/cTDaR_t00927.jpg'),cv2.COLOR_BGR2GRAY)
#showImage(img)
#showImage(get2img(img))
#showImage(get3img(img))

In [ ]:
def get2img(img):
    
    imgb = (img <= 128)*255
    imgb = imgb.astype(np.uint8)
    showImage(imgb)
    for _ in range(5):
        imgb = cv2.GaussianBlur(imgb,(5,5),-1)
        
    imgxy = cv2.Sobel(imgb,-1,dx=1,dy=1,ksize=5)
    
    imgt = imgxy
    imgxy = imgxy + imgb 

    for _ in range(3):
        imgxy = cv2.GaussianBlur(imgxy,(5,1),-1)

        imgxy = cv2.erode(imgxy,np.ones((3,5)),iterations = 2)
        imgxy = cv2.dilate(imgxy,np.ones((3,9)),iterations = 2)
        
        imgxy = ((imgxy > 15)*255).astype(np.uint8)

        #showImage(imgxy)

    #showImage(imgxy)
    return imgxy
#showImage(get2img(img))

In [ ]:
def get3img(img):
    imgb = (img <= 170)*255
    imgb = imgb.astype(np.uint8)
    
    imgb1 = (img <= 125)*255
    imgb1 = imgb1.astype(np.uint8)
    
    imgb = (imgb & imgb1) | imgb1
    
    for _ in range(5):
        imgb = cv2.erode(imgb,np.ones((1,3)),iterations = 1)
        imgb = cv2.GaussianBlur(imgb,(7,7),-1)
        imgb = ((imgb!=0)*255).astype(np.uint8)
    
    imgb = cv2.dilate(imgb,np.ones((3,7)),iterations = 3)
    
    imgb = (cv2.Canny(imgb,170,255)>15)*255
    imgb = imgb.astype(np.uint8)

    return imgb

#img = cv2.cvtColor(cv2.imread("./data/TRACKB_training/cTDaR_t00852.jpg"),cv2.COLOR_BGR2GRAY)
#showImage(img)
#showImage(get3img(img))

In [ ]:
#img1 = cv2.cvtColor(get2img(img),cv2.COLOR_GRAY2BGR)
#showImage(img1)

In [ ]:
from random import choice
from matplotlib.colors import cnames
from matplotlib.patches import Rectangle

colors = [c for c in cnames]

file = "../data/TRACKB_training/cTDaR_t00799.xml"
def get_color():
    return choice(colors)
def get_xy_cord(cord):
    cord = [list(map(int,xy.split(','))) for xy in cord]
    cord.sort()
    xy = cord[0]
    h,w =  abs(cord[-1][0] - xy[0]),abs(xy[1] - cord[-1][1])
    #print(xy,h,w)
    return np.asarray([xy,h,w])


#def flipHorizontal(img):
    
#def get_color():
rt = ET.parse(file).getroot()
imgname = "../data/TRACKB_training/" + rt.attrib['filename']
points = []
for table in rt:
    cord = table[0].attrib["points"].split()
    #if len(cord) != 4:
    #    continue
    points.append(get_xy_cord(cord))
    #for cell in table[1:]:
    #    cell_cord = cell[0].attrib['points'].split()
    #    #if len(cell_cord) == 4:
    #    #    continue
    #    points.append(get_xy_cord(cell_cord))
        
#img = cv2.imread(imgname)
#plot = showImage(img)

#shp_r = (1024,720)
#img_shape = img.shape[:2]#

#print(img_shape)
#imratio = min(shp_r)/min(img_shape)
#if imratio*max(img_shape) > max(shp_r):
#    imratio = max(shp_r)/max(img_shape)
#    
#print(imratio)

#im_shape = np.dot(imratio,img_shape).astype(np.int)

#print(im_shape)


#imr = cv2.resize(img,(im_shape[1],im_shape[0]),interpolation=cv2.INTER_CUBIC)
#print(imr.shape)

#imratio = 1

#plot = showImage(img)
#for xy,h,w in points:
#    plot.add_patch(Rectangle((xy[0]*imratio,xy[1]*imratio),h*imratio,w*imratio,edgecolor=get_color(),alpha=0.7,facecolor="none",linewidth=2))


In [ ]:
from random import shuffle
def get_color():
    return choice(colors)

mival = 0
colors = [c for c in cnames]
def get_mul_xy(xy):
    a,b = xy
    a,b = a+17,b+17
    return a*b
def get_xy_cord(cord,label:int = 0):
        cord = [list(map(int,xy.split(','))) for xy in cord]
        acord = [[get_mul_xy(xy),i ] for i,xy in enumerate(cord)]
        acord.sort()
        sindx,eindx = acord[0][1],acord[-1][1]
        xy = cord [sindx]
        w,h = abs(cord[eindx][0] - xy[0]),abs(xy[1] - cord[eindx][1])

        return np.asarray([max(0,xy[0]),max(0,xy[1]),max(0,w),max(0,h),label])

def get_XML_file_paths(path:str = "./*.xml"):
    """ Returns XML File PathList    """
    dirs = glob.glob(path)
    return dirs

#LABEL: 1:Table, 2:Cell

def get_xml_data(path:str = None):
    assert path != None, "Please give input path"
    global mival
    dli = get_XML_file_paths(path + "*.xml")
    ntable,ndocs,ncells = 0,0,0
    tables,img_path = [],[]
    for i,path_xml in enumerate(dli):
        tree = ET.parse(path_xml)
        root = tree.getroot()
        fname = root.attrib['filename']
        img_path.append(path + fname)
        ndocs += 1
        points = []
        for table in root:
            cord = table[0].attrib["points"].split()
            #_,_,h,w = get_xy_cord(cord,1)
            #if h < 1000 or w < 1000:
            #    print(path_xml,h,w)
            points.append(get_xy_cord(cord,1))
            ntable += 1
            for cell in table[1:]:
                cell_cord = cell[0].attrib['points'].split()
       #     #    ncells += 1
      #          get_xy_cord(cell_cord,2)
                points.append(np.asarray(get_xy_cord(cell_cord,2)))
                #break
        #shuffle(points)
        tables.append(np.asarray(points))
    
    print(ntable,ndocs,ncells,mival)
    return np.asarray(tables,dtype=object),img_path
data,img_path = get_xml_data("../data/TRACKB_training/")

In [ ]:
print(img_path[597])
img = cv2.imread(img_path[597])
lb = data[597]



plot = showImage(img)
print(img.shape)
for pg in lb:
    plot.add_patch(Rectangle((pg[0],pg[1]),pg[2],pg[3],edgecolor=get_color(),alpha=0.7,facecolor="none",linewidth=2))


In [ ]:
import pandas as pd
d2 = []
for image in data:
    for x,y,h,w in image:
        d2.append({'x':x,'y':y,'h':h,'w':w})


In [ ]:
pd1 = pd.DataFrame(d2)

pd1.describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.jointplot(x="w", y="h", data=pd1)
sns.jointplot(x="w", y="h", data=pd1, kind='kde')

In [ ]:
d3 = []
for path in img_path:
    shape = list(cv2.imread(path).shape[:2])
    d3.append({"h":shape[0],"w":shape[1]})
    

In [ ]:
pd2 = pd.DataFrame(d3)
pd2.describe()

In [ ]:

sns.jointplot(x="w", y="h", data=pd2)
sns.jointplot(x="w", y="h", data=pd2, kind='kde')

In [ ]:
from random import randint as rint

#ival = rint(0,len(img_path)-1)
ival = 78
impath = img_path[ival]
print(impath)
d5 =  data[ival]

In [ ]:
img = cv2.imread(impath)

In [ ]:
imgr = cv2.resize(img,(1080,1440))

In [ ]:

plot = showImage(imgr)
#for xy,h,w in points:
#pg = p3
#print(pg)
for pg in bxs[rint(20000,200000):]:
    plot.add_patch(Rectangle((pg[0],pg[1]),pg[2],pg[3],edgecolor=get_color(),alpha=0.7,facecolor="none",linewidth=2))


In [ ]:
d5

In [ ]:
p1 = d5[0][:4]

In [ ]:
s1 = img.shape[:2]
print(s1,p1)
p2 = p1[0]/s1[1],p1[1]/s1[0],p1[2]/s1[1],p1[3]/s1[0]

In [ ]:
print(p1)
print(p2)
p3 = p2[0]*1080,p2[1]*1440,p2[2]*1080,p2[3]*1440
print(p3)

In [ ]:
p1,s1,p3

In [ ]:
from importlib import reload
import  retina_net_model 

retina_net_model  = reload(retina_net_model)
AnchorBox = retina_net_model.AnchorBox
abxs = AnchorBox()
bxs = abxs.get_anchors(1300,800)

In [ ]:
len(bxs)

In [ ]:
import box_util
#box_util = reload(box_util)

b2xs = box_util.convert_to_min_max_corner(bxs)

In [ ]:
bix = []
def get_static_iou(b1,b2 = np.asarray([358.0,274.0,804.0,1275.0])):
    mic = min(b1[:2],b2[:2])
    mxc = max(b1[2:],b2[2:])
    
for i,row in enumerate(b2xs):
    srow = row.numpy()
    x,y,w,h = srow
    if 300.0 < x < 400.0 and 200 < y < 300:
        print([x,y,w,h])
    #bix.append([g[2]*g[3],i])

In [ ]:
bix.sort()

In [ ]:
for row in bxs.numpy():
    print(row)

In [ ]:
data[0]

In [ ]:
vals,counts = np.unique(img,return_counts=True)
fig, ax = plt.subplots(figsize=(100,20))
sns.barplot(x=vals,y=counts,ax=ax)

In [ ]:
sval = 0
for v,c in zip(vals,counts):
    sval += (v*c)

print(sval/(len(vals)*len(counts)*256))

In [ ]:
pth = "./data/TRACKB_training/*.jpg"
li = glob.glob(pth)
color_dict = dict()
for i in range(256):
    color_dict[i] = 0
    
from multiprocessing import Pool
def some_func(imgp:str):
    print(".",end="")
    img = cv2.cvtColor(cv2.imread(imgp),cv2.COLOR_BGR2GRAY)
    vals,counts = np.unique(img,return_counts=True)
    for i,v in enumerate(vals):
        color_dict[v] = counts[i]    


In [ ]:
for imgp in li:
    print(".", end = "")
    some_func(imgp)

In [ ]:
vals = [i for i in color_dict]
counts = [color_dict[i] for i in color_dict]
fig, ax = plt.subplots(figsize=(100,20))
sns.barplot(x=vals,y=counts,ax=ax)

In [ ]:
def get_all_three_images(img_path:str):
    img = cv2.imread(img_path)
    img_gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(get2img(img_gry),cv2.COLOR_GRAY2BGR)
    img3 = cv2.cvtColor(get3img(img_gry),cv2.COLOR_GRAY2BGR)
    return [img,img2,img3]

In [ ]:
print(img_path[0])
imgs = get_all_three_images(img_path[0])

In [ ]:
import ImageUtil
import Box_Util
imutil = reload(ImageUtil)
bxutil = reload(Box_Util)


for i in range(1,2):
    print("\n\n\n\t\t\t\t\t\t",i)
    d = ImageUtil.preprocess_image(get_all_three_images(img_path[i]),data[i])

    
plt1 = showImage(d[0][1])
x,y,w,h = bxutil.convert_to_display_format(d[1][234])
#x,y,w,h = int(x),int(y),int(w),int(h)
print(x,y,w,h)
#w,h = 668,312
plt1.add_patch(Rectangle((x,y),h,w,edgecolor='gray',alpha=0.7,facecolor="none",linewidth=2))

In [ ]:
d[1]

In [ ]:
import RetinaModel

rm = reload(RetinaModel)


lb = rm.LabelEncoder()
imgs,img_lb = [],[]
for i in range(32):
    imgs.append(get_all_three_images(img_path[i]))
    img_lb.append(data[i])

In [ ]:
data = lb.encode_batch(imgs,img_lb)

In [ ]:
data[0][234]

In [ ]:
from random import randint as rint

In [ ]:
import retina_net_model as rmodel
import tensorflow as tf
from matplotlib.patches import Rectangle
rmodel = reload(rmodel)
abox = rmodel.AnchorBox()
img = cv2.cvtColor(cv2.imread(img_path[1]),cv2.COLOR_BGR2GRAY)
abbb = tf.cast(abox.get_anchors(img.shape[0],img.shape[1]),dtype = tf.int32)
len(abbb)


In [ ]:
bbox = abbb[...,2]*abbb[...,3]
bbox = bbox.numpy()

ival = []
for i,val in enumerate(bbox):
    ival.append([val,i])

In [ ]:
ival.sort()

In [ ]:
shuffle(ival)

In [ ]:
plot = showImage(img)

for i in range(rint(500,1000)):
    x,y,w,h = abbb[ival[-1*i][1]].numpy()
    #print([x,y,w,h],ival[-1*i],w*h)
    plot.add_patch(Rectangle((x,y),h,w,edgecolor=get_color(),alpha=0.7,facecolor="none",linewidth=2))

In [ ]:
from time import time

import image_util  as imutil
from importlib import reload
reload(imutil)
imaug = imutil.ImageAugmenter()

t1 = time()
imgs = imaug.get_all_3_images("./data/custom_data/cbc1.png")
t2 = time()
print(t2-t1)


showImage(imgs[0])
showImage(imgs[1])
showImage(imgs[2])